In [9]:
import numpy as np

from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.manifold import TSNE
from gensim.models import KeyedVectors
import pandas as pd
from keras.layers import LSTM
import joblib

EMBEDDING_DIMENSION = 100
VOCABULARY_SIZE = 150443

def read_corpus_dataset(dataset_path):
    data = pd.read_csv(dataset_path, error_bad_lines=False)
    labels = data['sentiment']
    tweets = data['tweet']

    return tweets, labels

In [15]:
def tokenize_dataset(tweets, vocabulary_size):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(tweets)
    sequences = tokenizer.texts_to_sequences(tweets)
    # dictionary word:index
    word_indices = {}
    for key, value in tokenizer.word_index.items():
        word_indices[key] = value
        if value == vocabulary_size:
            break
    joblib.dump(tokenizer,"tokenizer.joblib")
    
    
    print('===> Number of words in dataset: {}'.format(len(word_indices)))

    return sequences, word_indices

def read_embeddings():
    GLOVE_PATH = "/home/smartcat/Documents/GitHub/glove.twitter.27B.100d.txt"
    embeddings = {}
    with open(GLOVE_PATH) as glove_file:
        for line in glove_file:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings[word] = coefs
    print('===> Using {} embedding vectors\n'.format(len(embeddings)))
    return embeddings


In [11]:

# mapping pretrained coefficients to dataset
def create_embedding_matrix(word_indices, embeddings):
    WORDS_NUM = len(word_indices) + 1
    embedding_matrix = np.zeros((WORDS_NUM, EMBEDDING_DIMENSION))
    for word, i in word_indices.items():
        word_vector = embeddings.get(word)
        if word_vector is not None:
            embedding_matrix[i] = word_vector

    return WORDS_NUM, embedding_matrix


def get_model(WORDS_NUM, embedding_matrix, MAX_SEQUENCE_LENGTH):
    model = Sequential()
    model.add(Embedding(WORDS_NUM,
                        EMBEDDING_DIMENSION,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=False))
    model.add(Conv1D(256, 3, padding='same', activation='relu'))
    model.add(Conv1D(128, 3, padding='same', activation='relu'))
    model.add(Conv1D(64, 3, padding='same'))
    model.add(GlobalMaxPooling1D())
    # model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(180, activation='sigmoid'))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    return model

def get_model_LSTM(WORDS_NUM, embedding_matrix, MAX_SEQUENCE_LENGTH):
    model = Sequential()
    model.add(Embedding(WORDS_NUM,
                        EMBEDDING_DIMENSION,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=False))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [16]:
print('===> Reading GloVe words embeddings\n')
embeddings = read_embeddings()

===> Reading GloVe words embeddings

===> Using 1193514 embedding vectors



In [17]:
tweets, labels = read_corpus_dataset("new_data.csv")
sequences, word_indices = tokenize_dataset(tweets, VOCABULARY_SIZE)

===> Number of words in dataset: 150443


In [6]:
# (train_summary, train_review_text, train_labels), (test_summary, test_review_text, test_labels) = load_amazon_smaller()
# (X_train, y_train), (X_test, y_test) = (train_summary, train_labels), (test_summary, test_labels)

In [18]:
MAX_SEQUENCE_LENGTH = len(max(sequences, key=lambda x: len(x)))
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [19]:
# labels = to_categorical(labels)
print('===> Data shape: {}'.format(padded_sequences.shape))
print('===> Labels shape: {}'.format(labels.shape))
labels[labels==4]=1

===> Data shape: (200000, 46)
===> Labels shape: (200000,)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(padded_sequences,
                                                        labels,
                                                        test_size=0.2)

In [21]:
WORDS_NUM, embedding_matrix = create_embedding_matrix(word_indices,
                                                          embeddings)

In [22]:
model = get_model(WORDS_NUM, embedding_matrix, MAX_SEQUENCE_LENGTH)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 46, 100)           15044400  
_________________________________________________________________
conv1d (Conv1D)              (None, 46, 256)           77056     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 128)           98432     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 64)            24640     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 180)               1

In [23]:
model.fit(X_train,
              y_train,
              batch_size=128,
              epochs=5,
              validation_data=(X_test, y_test),
              verbose=1)

Epoch 1/5
 348/1250 [=======>......................] - ETA: 7s - loss: 0.5572 - accuracy: 0.7090

KeyboardInterrupt: 

In [13]:
pred = model.predict(X_test)
pred[pred<0.5] = 0
pred[pred!=0] =1

In [28]:
from sklearn.metrics import classification_report
def save_model_and_weights(model):
    print('===> Saving model and weights\n')
    model_json = model.to_json()
    with open("model.json", 'w') as json_file:
        json_file.write(model_json)

    model.save_weights("model.h5")
save_model_and_weights(model)

===> Saving model and weights



In [16]:
print(classification_report(y_test, pred))
model.save("CNN.h5")

              precision    recall  f1-score   support

           0       0.83      0.76      0.79     19958
           1       0.78      0.85      0.81     20042

    accuracy                           0.80     40000
   macro avg       0.80      0.80      0.80     40000
weighted avg       0.80      0.80      0.80     40000



In [27]:
model = get_model_LSTM(WORDS_NUM, embedding_matrix, MAX_SEQUENCE_LENGTH)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 46, 100)           15044400  
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 15,161,777
Trainable params: 117,377
Non-trainable params: 15,044,400
_________________________________________________________________


In [ ]:
model.fit(X_train,
              y_train,
              batch_size=128,
              epochs=5,
              validation_data=(X_test, y_test),
              verbose=1)

Epoch 1/5
1250/1250 [==============================] - 11s 9ms/step - loss: 0.4949 - accuracy: 0.7587 - val_loss: 0.4590 - val_accuracy: 0.7822
Epoch 2/5
1250/1250 [==============================] - 11s 9ms/step - loss: 0.4494 - accuracy: 0.7893 - val_loss: 0.4468 - val_accuracy: 0.7885
Epoch 3/5
1250/1250 [==============================] - 11s 9ms/step - loss: 0.4315 - accuracy: 0.7997 - val_loss: 0.4287 - val_accuracy: 0.7987
Epoch 4/5
 421/1250 [=========>....................] - ETA: 6s - loss: 0.4174 - accuracy: 0.8065

In [ ]:
pred = model.predict(X_test)
pred[pred<0.5] = 0
pred[pred!=0] =1

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, pred))

In [13]:
from keras import models
model = models.load_model("CNN.h5")

In [14]:
tokenizer = joblib.load("tokenizer.joblib")
tweets = ["Very good"]
sequences = tokenizer.texts_to_sequences(tweets)
padded_sequences = pad_sequences(sequences, maxlen=46)
pred = model.predict(padded_sequences)
print(pred)

[[0.90275687]]


In [16]:
print(pred[0][0])

0.90275687


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 46, 100)           15044400  
_________________________________________________________________
conv1d (Conv1D)              (None, 46, 256)           77056     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 46, 128)           98432     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 64)            24640     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 180)               1

In [4]:
import joblib

In [8]:
joblib.dump(model,"model.joblib")

TypeError: can't pickle _thread.RLock objects